In [0]:
import pandas as pd
import networkx as nx

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

In [0]:
#filepath="/content/gdrive/My Drive/Network Analytics/data/"
filepath=''

In [0]:
!pip install pyexcel_ods

In [0]:
transition_matrix=pd.read_excel(filepath+"transition_matrix.ods", engine="odf")
transition_matrix=transition_matrix.drop(transition_matrix[transition_matrix["Unnamed: 0"].isnull()].index)
transition_matrix.index=transition_matrix["Unnamed: 0"]
transition_matrix=transition_matrix.drop(["Unnamed: 0"], axis=1)
transition_matrix.fillna(0,inplace=True)
transition_matrix.shape

In [0]:
transition_matrix.head()

In [0]:

list(transition_matrix.columns)

In [0]:
import numpy as np
  # initialize the population vector from the origin-destination flow matrix
number_neighbourhoods = 59
population_neighbourhoods  = np.random.randint(10000, 25000, number_neighbourhoods)  # number of locations
# initialize the S group with the respective populations


In [0]:
# graph = nx.DiGraph()
# graph.add_nodes_from(list(transition_matrix.columns))
# graph=nx.from_numpy_matrix(OD_matrix_AN)


In [0]:
## inititialise the infections 
SIR = np.zeros(shape=(number_neighbourhoods , 3)) # make a numpy array with 3 columns for keeping track of the S, I, R groups
SIR[:,0] = population_neighbourhoods 
infections_initial = np.random.randint(1, 10, number_neighbourhoods)
OD_matrix_AN = transition_matrix.copy().to_numpy()

# change to OD matrix:
for i in range(number_neighbourhoods):
    transition_numbers = np.random.randint(100, 1000, number_neighbourhoods)
    OD_matrix_AN[:,i] = OD_matrix_AN[:,i] * transition_numbers

In [0]:


# change the transition matrix of people in function of the problem that you are trying to solve 
#experiment_case = 'initial'
experiment_case = 'reduce_imp_node'
if experiment_case == 'initial':
    pass
elif experiment_case == 'reduce_imp_node':
    graph = nx.DiGraph()
    graph.add_nodes_from(list(transition_matrix.columns))
    graph=nx.from_numpy_matrix(OD_matrix_AN)
    centrality_dict = nx.degree_centrality(graph)
    sorted_x = sorted(centrality_dict.items(), key=lambda x: x[1])
    best_ones = sorted_x[-10:] 
    print(len(best_ones))
    
    # stop people from going in and out on the best ones --max 10 ?
    for idx, value in best_ones:
        max_people = np.random.randint(0, 15)
        OD_matrix_AN[:, idx] = [0]*number_neighbourhoods
# else: ## case 2
#     for idx in range(number_neighbourhoods):
#         max_people = 250
#         OD_matrix_AN[:, idx] = np.where(OD_matrix_AN[idx,:] > 0, max_people, 0)

SIR[:, 0] = [int(el - infections_initial[i]) for i, el in enumerate(SIR[:, 0] )]
SIR[:, 1] = [int(el + infections_initial[i]) for i, el in enumerate(SIR[:, 1] )]                     


In [0]:
infections_initial

In [0]:
OD_matrix_AN

In [0]:
from tqdm import tqdm_notebook
# row normalize the SIR matrix for keeping track of group proportions
row_sums = SIR.sum(axis=1)
SIR_n = SIR / row_sums[:, np.newaxis]

# initialize parameters
number_days = 200
beta = 1.07
gamma = 1/3.6
public_trans = 0.5                               # alpha
R0 = beta/gamma
beta_vec = np.random.gamma(1.6, 2, locs_len)
gamma_vec = np.full(locs_len, gamma)
public_trans_vec = np.full(locs_len, public_trans)

# make copy of the SIR matrices 
SIR_sim = SIR.copy()
SIR_nsim = SIR_n.copy()

# run model
print(SIR_sim.sum(axis=0).sum() == N_k.sum())
infected_pop_norm = []
susceptible_pop_norm = []
recovered_pop_norm = []

for time_step in tqdm_notebook(range(number_days)):
    infected_mat = np.array([SIR_nsim[:,1],]*locs_len).transpose()
    OD_infected = np.round(OD_matrix_AN*infected_mat)
    inflow_infected = OD_infected.sum(axis=0)
    inflow_infected = np.round(inflow_infected*public_trans_vec)
    print('total infected inflow: ', inflow_infected.sum())
    new_infect = beta_vec*SIR_sim[:, 0]*inflow_infected/(N_k + OD_matrix_AN.sum(axis=0))
    new_recovered = gamma_vec*SIR_sim[:, 1]
    new_infect = np.where(new_infect>SIR_sim[:, 0], SIR_sim[:, 0], new_infect)
    SIR_sim[:, 0] = SIR_sim[:, 0] - new_infect
    SIR_sim[:, 1] = SIR_sim[:, 1] + new_infect - new_recovered
    SIR_sim[:, 2] = SIR_sim[:, 2] + new_recovered
    SIR_sim = np.where(SIR_sim<0,0,SIR_sim)
    # recompute the normalized SIR matrix
    row_sums = SIR_sim.sum(axis=1)
    SIR_nsim = SIR_sim / row_sums[:, np.newaxis]
    S = SIR_sim[:,0].sum()/N_k.sum()
    I = SIR_sim[:,1].sum()/N_k.sum()
    R = SIR_sim[:,2].sum()/N_k.sum()
    print(S, I, R, (S+I+R)*N_k.sum(), N_k.sum())
    print('\n')
    infected_pop_norm.append(I)
    susceptible_pop_norm.append(S)
    recovered_pop_norm.append(R)


In [0]:
import matplotlib.pyplot as plt
days_array = range(number_days)
plt.plot(days_array, infected_pop_norm, label='Infected Population')
plt.plot(days_array, susceptible_pop_norm, label='Susceptible Population')
plt.plot(days_array, recovered_pop_norm, label='Recovered Population')
plt.legend(loc = 1)
plt.xlabel('Day number')
plt.ylabel('Population Fraction')
plt.title('Evolution of number of SIR over time~')
